In [1]:
import json
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd
import pandas as pd

In [2]:
notebooks_folder = Path('.').absolute()

In [3]:
root_folder = notebooks_folder.parent

In [4]:
cable_zip = ZipFile((notebooks_folder / 'source_files' / 'mygeodata.zip'))

In [5]:
cable_json = root_folder / 'app' / 'api' / 'rsn-cable.json'

In [6]:
default_properties = {
    "reference_designator": "RS",
    "array_name": "Regional Scale Node",
    "name":"Backbone Cable",
    "description":"Approximately 900 km of primary backbone electro-optical cable have been installed from the Shore Station to the seven Primary Node locations spanning the Juan de Fuca Plate. The cable provides power (10 kV DC, up to 8 A) and communications (10 Gb/s) via fiber-optics and copper between the Primary Nodes and the Shore Station in Pacific City, Oregon.",
    "image_url":"https://s3.us-west-2.amazonaws.com/media.ooica.net/wp-content/uploads/2019/01/03192635/cableacrosscollapse3-300x169.jpg",
    "wp_url":"https://interactiveoceans.washington.edu/technology/cables-connectors/"
}

In [7]:
cables_gdf = []
for zf in filter(lambda f: f.filename.endswith('.geojson'), cable_zip.filelist):
    file_name = Path(zf.filename).name.lower()
    # Only looks for cable in file name and ignore buried cables.
    if ('cable' in file_name) and ('buried' not in file_name):
        print(file_name)
        geojson_file = cable_zip.open(zf.filename)
        gdf = gpd.read_file(geojson_file)
        gdf['cable_id'] = 'RSN-Cable'
        cables_gdf.append(gdf)

ce02shsm_riser_cable_alternate_site.geojson
ce04ossm_riser_cable_alternate_site.geojson
cables.geojson
mooring_riser_cable.geojson
mooring_riser_cable_dp.geojson
mooring_riser_cable_sp.geojson
mooring_riser_cables.geojson
moorings_cabled.geojson
moorings_non_cabled.geojson
primary_cable.geojson
secondary_cable.geojson
secondary_cable_oily.geojson


In [8]:
cable_gdf = pd.concat(cables_gdf).reset_index(drop=['index']).dissolve(by='cable_id')[['geometry']]

In [9]:
for k,v in default_properties.items():
    cable_gdf[k] = v

In [10]:
cable_json.write_text(json.dumps(next(cable_gdf.iterfeatures()), indent=2), encoding='utf-8')

189343